In [2]:
from datasets import load_dataset

dataset = load_dataset("dair-ai/emotion")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
from transformers import AutoTokenizer
model_name="togethercomputer/Llama-2-7B-32K-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, DataCollatorWithPadding
llama_tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token
# col_to_delete = ['idx']
col_to_delete = ['text']

def llama_preprocessing_function(examples):
    return llama_tokenizer(examples['text'], truncation=True, max_length=350)

llama_tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
llama_tokenized_datasets.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
llama_data_collator = DataCollatorWithPadding(tokenizer=llama_tokenizer)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup

import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

In [8]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_name)
config

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

LlamaConfig {
  "_name_or_path": "togethercomputer/Llama-2-7B-32K-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoModelForCausalLM": "togethercomputer/LLaMA-2-7B-32K--modeling_flash_llama.LlamaForCausalLM"
  },
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 11008,
  "max_position_embeddings": 32768,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": {
    "factor": 8.0,
    "type": "linear"
  },
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.36.2",
  "use_cache": true,
  "vocab_size": 32000
}

In [9]:
config._name_or_path=model_name
config.hidden_size=4096
config.num_hidden_layers=32
config.n_head=32
config.num_labels=6
config.pad_token_id=tokenizer.pad_token_id
config.hidden_dropout = 0.1
config.transform=False
config.text='Classify the joy or sadness or anger or fear or love or surprise from the text:'

In [10]:
from llamaSKT import  PromptForSequenceClassification

model = PromptForSequenceClassification.from_pretrained(
    model_name,
    config=config,

)

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prompt sequence length:  20


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of PromptForSequenceClassification were not initialized from the model checkpoint at togethercomputer/Llama-2-7B-32K-Instruct and are newly initialized: ['transformer.layers.31.mlp.gate_proj.weight', 'transformer.layers.0.mlp.up_proj.weight', 'transformer.layers.14.self_attn.k_proj.weight', 'transformer.layers.18.mlp.gate_proj.weight', 'transformer.layers.28.self_attn.v_proj.weight', 'transformer.layers.3.mlp.down_proj.weight', 'transformer.layers.5.self_attn.v_proj.weight', 'transformer.layers.3.mlp.up_proj.weight', 'transformer.layers.5.mlp.up_proj.weight', 'transformer.layers.9.self_attn.q_proj.weight', 'transformer.layers.16.mlp.up_proj.weight', 'transformer.layers.22.self_attn.q_proj.weight', 'transformer.layers.22.mlp.up_proj.weight', 'transformer.layers.13.self_attn.v_proj.weight', 'transformer.layers.20.self_attn.q_proj.weight', 'transformer.layers.23.input_layernorm.weight', 'transformer.layers.7.self_attn.v_proj.weight', 'transformer.layers.24.mlp.gate_proj.weigh

In [11]:
# Total number of parameters in the model
total_parameters = model.num_parameters()

# Total number of trainable parameters in the model
trainable_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Calculate the percentage of trainable parameters
percentage_trainable = (trainable_parameters / total_parameters) * 100

print(f"Total Parameters: {total_parameters}")
print(f"Trainable Parameters: {trainable_parameters}")
print(f"Percentage Trainable: {percentage_trainable:.20f}%")

Total Parameters: 6607450118
Trainable Parameters: 106502
Percentage Trainable: 0.00161184720426216319%


In [12]:
import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)

    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [13]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./r_task',
    #learning_rate=0.001,
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=5,
 

    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=500,
    logging_steps=500,
   
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=llama_tokenized_datasets["train"],
    eval_dataset=llama_tokenized_datasets["validation"],
    tokenizer=llama_tokenizer,
    data_collator=llama_data_collator,
    compute_metrics=compute_metrics
)

trainer.train()


Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
500,1.589200,1.529009,0.306554,0.189375,0.131407,0.387000
1000,1.513200,1.528721,0.166753,0.248930,0.189677,0.457000
1500,1.456700,1.435503,0.253892,0.283456,0.235081,0.517000
2000,1.417700,1.372740,0.524523,0.287132,0.240526,0.527500
2500,1.373700,1.311002,0.424610,0.358360,0.352733,0.559000
3000,1.297200,1.230281,0.587255,0.303663,0.262452,0.544500
3500,1.188100,1.183529,0.530947,0.329618,0.304548,0.565000
4000,1.161900,1.108806,0.680794,0.402476,0.413029,0.609500
4500,1.091700,1.063190,0.579622,0.481217,0.490156,0.634500
5000,1.018100,0.963616,0.675307,0.472644,0.481714,0.663500


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=13335, training_loss=0.9647529696452738, metrics={'train_runtime': 12417.1072, 'train_samples_per_second': 6.443, 'train_steps_per_second': 1.074, 'total_flos': 1.2685136906133478e+17, 'train_loss': 0.9647529696452738, 'epoch': 5.0})

In [14]:
trainer.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
500,0.662300,0.682933,0.829779,0.627194,0.676155,0.772000
1000,0.652400,0.629889,0.801554,0.659494,0.698544,0.785000
1500,0.647200,0.611979,0.806290,0.712392,0.747228,0.810500
2000,0.612400,0.627952,0.813254,0.734793,0.761125,0.824500
2500,0.618600,0.593918,0.804059,0.732576,0.755907,0.812500
3000,0.565200,0.558774,0.814126,0.743324,0.771783,0.830500
3500,0.533400,0.648206,0.837385,0.666490,0.718403,0.797000
4000,0.567700,0.571160,0.835500,0.733272,0.771744,0.828500
4500,0.523300,0.506570,0.829764,0.749666,0.777955,0.836500
5000,0.540600,0.491959,0.844016,0.745867,0.777274,0.851000


Could not locate the best model at ./r_task/checkpoint-12500/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=13335, training_loss=0.50433715431262, metrics={'train_runtime': 12553.2084, 'train_samples_per_second': 6.373, 'train_steps_per_second': 1.062, 'total_flos': 1.2685136906133478e+17, 'train_loss': 0.50433715431262, 'epoch': 5.0})